In [178]:
import torch 
import lightning as L
import torchmetrics 
from scipy.io import loadmat
import matplotlib.pyplot as plt 
import numpy as np
import pandas as pd
from pathlib import Path
import scipy
import itertools
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error
import pandas as pd
from statsmodels.tsa.stattools import adfuller, grangercausalitytests
from statsmodels.tsa.api import VAR
from statsmodels.tsa.ar_model import AutoReg
from sklearn.metrics import mean_squared_error
from tqdm import tqdm
from sklearn.feature_selection import mutual_info_regression

In [ ]:
#%%
def load_wildppg_participant(path):
    """
    Loads the data of a WildPPG participant and cleans it to receive nested dictionaries
    """
    loaded_data = scipy.io.loadmat(path)
    loaded_data['id'] = loaded_data['id'][0]
    if len(loaded_data['notes'])==0:
        loaded_data['notes']=""
    else:
        loaded_data['notes']=loaded_data['notes'][0]

    for bodyloc in ['sternum', 'head', 'wrist', 'ankle']:
        bodyloc_data = dict() # data structure to feed cleaned data into
        sensors = loaded_data[bodyloc][0].dtype.names
        for sensor_name, sensor_data in zip(sensors, loaded_data[bodyloc][0][0]):
            bodyloc_data[sensor_name] = dict()
            field_names = sensor_data[0][0].dtype.names
            for sensor_field, field_data in zip(field_names, sensor_data[0][0]):
                bodyloc_data[sensor_name][sensor_field] = field_data[0]
                if sensor_field == 'fs':
                    bodyloc_data[sensor_name][sensor_field] = bodyloc_data[sensor_name][sensor_field][0]
        loaded_data[bodyloc] = bodyloc_data
    return loaded_data

def panPeakDetect(detection, fs):
    """
    Jiapu Pan and Willis J. Tompkins.
    A Real-Time QRS Detection Algorithm.
    In: IEEE Transactions on Biomedical Engineering
    BME-32.3 (1985), pp. 230–236.

    Original implementation by Luis Howell luisbhowell@gmail.com, Bernd Porr, bernd.porr@glasgow.ac.uk, DOI: 10.5281/zenodo.3353396
    """
    min_distance = int(0.25 * fs)

    signal_peaks = [0]
    noise_peaks = []

    SPKI = 0.0
    NPKI = 0.0

    threshold_I1 = 0.0
    threshold_I2 = 0.0

    RR_missed = 0
    indexes = []

    missed_peaks = []
    peaks = scipy.signal.find_peaks(detection,distance=min_distance)[0]

    thres_weight = 0.125

    for index, peak in enumerate(peaks):

        if peak>4*fs and threshold_I1>max(detection[peak-4*fs:peak]): # reset thresholds if we do not see any peaks anymore
            SPKI_n = max(detection[peak-4*fs:peak])
            NPKI = min(NPKI*SPKI_n/SPKI, np.percentile(detection[peak-4*fs:peak], 80))
            SPKI = SPKI_n
            threshold_I1 = NPKI + 0.25 * (SPKI - NPKI)
            threshold_I2 = 0.5 * threshold_I1



        if detection[peak] > threshold_I1 and (peak - signal_peaks[-1]) > 0.3 * fs:
            signal_peaks.append(peak)
            indexes.append(index)
            SPKI = thres_weight * detection[signal_peaks[-1]] + (1-thres_weight) * SPKI
            if RR_missed != 0:
                if signal_peaks[-1] - signal_peaks[-2] > RR_missed:
                    missed_section_peaks = peaks[indexes[-2] + 1:indexes[-1]]
                    missed_section_peaks2 = []
                    for missed_peak in missed_section_peaks:
                        if missed_peak - signal_peaks[-2] > min_distance and signal_peaks[
                            -1] - missed_peak > min_distance and detection[missed_peak] > threshold_I2:
                            missed_section_peaks2.append(missed_peak)

                    if len(missed_section_peaks2) > 0:
                        signal_missed = [detection[i] for i in missed_section_peaks2]
                        index_max = np.argmax(signal_missed)
                        missed_peak = missed_section_peaks2[index_max]
                        missed_peaks.append(missed_peak)
                        signal_peaks.append(signal_peaks[-1])
                        signal_peaks[-2] = missed_peak
            if len(signal_peaks)>100 and thres_weight>0.1:
                thres_weight = 0.0125


        else:
            noise_peaks.append(peak)
            NPKI = thres_weight * detection[noise_peaks[-1]] + (1-thres_weight) * NPKI

        threshold_I1 = NPKI + 0.25 * (SPKI - NPKI)
        threshold_I2 = 0.5 * threshold_I1

        if len(signal_peaks) > 8:
            RR = np.diff(signal_peaks[-9:])
            RR_ave = int(np.mean(RR))
            RR_missed = int(1.66 * RR_ave)

    signal_peaks.pop(0)

    return signal_peaks


def pan_tompkins_detector(unfiltered_ecg, sr):
    """
    Jiapu Pan and Willis J. Tompkins.
    A Real-Time QRS Detection Algorithm.
    In: IEEE Transactions on Biomedical Engineering
    BME-32.3 (1985), pp. 230–236.

    Original implementation by Luis Howell luisbhowell@gmail.com, Bernd Porr, bernd.porr@glasgow.ac.uk, DOI: 10.5281/zenodo.3353396
    """
    maxQRSduration = 0.150  # sec
    filtered_ecg = butter_bandpass_filter(unfiltered_ecg, 5, 15, sr, order=1)

    diff = np.diff(filtered_ecg)
    squared = diff * diff

    mwa = scipy.ndimage.uniform_filter1d(squared, size=int(maxQRSduration * sr))
    # cap mwa during motion artefacts to make sure it does not screw the thresholds
    maxvals = scipy.ndimage.maximum_filter1d(filtered_ecg, size=int(maxQRSduration * sr))[:-1]/400
    mwa = np.asarray([v if v < maxval else maxval for maxval, v in zip(maxvals, mwa)])

    mwa[:int(maxQRSduration * sr * 2)] = 0

    searchr = int(maxQRSduration * sr)
    peakfind = butter_bandpass_filter(unfiltered_ecg, 7.5, 20, sr, order=1)

    mwa_peaks = panPeakDetect(mwa, sr)
    r_peaks2 = []
    for rp in mwa_peaks:
        r_peaks2.append(rp - searchr + np.argmax(peakfind[rp - searchr:rp + searchr + 1]))
    r_peaks3 = []
    for rp in r_peaks2:
        r_peaks3.append(rp - 2 + np.argmax(unfiltered_ecg[rp - 2:rp + 3])) # adjust by at most 2 samples to hit raw data max
    return np.asarray(r_peaks3)


def quotient_filter(hbpeaks, outlier_over=5, sampling_rate=128, tol=0.8):
    '''
    Function that applies a quotient filter similar to what is described in
    "Piskorki, J., Guzik, P. (2005), Filtering Poincare plots"
    it preserves peaks that are part of a sequence of [outlier_over] peaks with
    a tolerance of [tol]'''
    good_hbeats = []
    good_rrs = []
    good_rrs_x = []
    for i, peak in enumerate(hbpeaks[:-(outlier_over-1)]):
        hb_intervals = [hbpeaks[j]-hbpeaks[j-1]  for j in range(i+1, i+outlier_over)]
        hr = 60/((sum(hb_intervals))/((outlier_over-1)*sampling_rate))
        if min(hb_intervals) > max(hb_intervals)*tol and hr > 35 and hr < 185: # -> good data

            for p in hbpeaks[i:i+outlier_over]:
                if len(good_hbeats) == 0 or p > good_hbeats[-1]:
                    good_hbeats.append(p)
                    if len(good_hbeats) > 1:
                        rr = good_hbeats[-1]-good_hbeats[-2]
                        if rr<min(hb_intervals)/tol and rr>max(hb_intervals)*tol:
                            good_rrs.append(rr)
                            good_rrs_x.append((good_hbeats[-1]+good_hbeats[-2])/2)
    return np.array(good_hbeats), np.array(good_rrs), np.array(good_rrs_x)

def butter_bandpass(lowcut, highcut, fs, order=5):
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    sos = scipy.signal.butter(order, [low, high], analog=False, btype='band', output='sos')
    return sos

def butter_bandpass_filter(data, lowcut, highcut, fs, order=5):
    sos = butter_bandpass(lowcut, highcut, fs, order=order)
    y = scipy.signal.sosfiltfilt(sos, data)
    return y


In [ ]:
def process_data(winsize: int = 8):
    all_hrs = []
    p = Path("C:/Users/cleme/ETH/Master/Thesis/data/WildPPG/data/WildPPG_Part_an0.mat")

    part_data = load_wildppg_participant(p.absolute())
    import pdb 
    pdb.set_trace()
    x = part_data["ankle"]["acc_x"]["v"]
    y = part_data["ankle"]["acc_y"]["v"]
    z = part_data["ankle"]["acc_z"]["v"]
    imu = np.sqrt(x**2 + y**2 + z**2)

    r_peaks = pan_tompkins_detector(part_data['sternum']['ecg']['v'], part_data['sternum']['ecg']['fs'])
    ecgpks_filt, rrs, rrxs = quotient_filter(r_peaks, outlier_over=5, tol=0.75)

    hrs = []
    imus = []
    for win_s in range(0, max(ecgpks_filt), winsize * part_data['sternum']['ecg']['fs']):
        rr_in_win = rrs[np.logical_and(rrxs > win_s, rrxs < win_s + winsize * part_data['sternum']['ecg']['fs'])]
        window_imu = imu[win_s: win_s + winsize * part_data['sternum']['ecg']['fs']]
        imus.append(np.mean(window_imu))
        if len(rr_in_win) > 1:  # at least 2
            hrs.append(60 * len(rr_in_win) / (np.sum(rr_in_win) / part_data['sternum']['ecg']['fs']))
        else:
            hrs.append(0) # invalid / noisy ecg
    all_hrs.append([[h] for h in hrs]) # ground truth hr in 8s windows, make col vec

    outdict = {"data_bpm_values":np.asanyarray(all_hrs, dtype=object), "imu": np.array(imus)}
    return outdict

In [ ]:
def make_supervised_windows(hr, imu, L=30, H=3, imu_lags=1):
    """
    hr, imu: 1D numpy arrays (same length) for a single subject/sequence
    L: lookback length used as HR_past
    H: forecast horizon steps ahead (e.g., H=1,3,5,...)
    imu_lags: number of causal IMU lags to include (1 => current t)

    Returns:
      Z: (N, L)   HR past (t-L+1 ... t)
      X: (N, imu_lags)  IMU lags (t, t-1, ..., t-imu_lags+1)
      y: (N,)  HR future at t+H
    """
    assert hr.ndim == 1 and imu.ndim == 1 and len(hr) == len(imu)
    T = len(hr)
    max_lag = max(L, imu_lags - 1)
    # last usable t is T-H-1 ; first usable t is max_lag
    t_start = max_lag
    t_end = T - H - 1
    if t_end < t_start:
        return np.empty((0, L)), np.empty((0, imu_lags)), np.empty((0,))

    idx = np.arange(t_start, t_end + 1)
    N = len(idx)

    # HR past Z
    Z = np.stack([hr[idx - k] for k in range(L, 0, -1)], axis=1)  # shape (N, L)

    # IMU lags X (current to past)
    X = np.stack([imu[idx - k] for k in range(imu_lags)], axis=1)  # (N, imu_lags)

    # target y = future HR at t+H
    y = hr[idx + H]
    return Z, X, y


def conditional_mutual_info(
    hr, imu, L=30, H=3, imu_lags=1, n_neighbors=5, random_state=0
):
    """
    Estimate I(IMU ; HR_future | HR_past) via difference of mutual infos:
      I([IMU,Z]; y) - I(Z; y)
    using sklearn's mutual_info_regression (kNN estimator).
    Returns: cmi_estimate (nats), N_effective
    """
    Z, X, y = make_supervised_windows(hr, imu, L=L, H=H, imu_lags=imu_lags)
    if len(y) == 0:
        return np.nan, 0

    # Concatenate features
    ZX = np.concatenate([Z, X], axis=1)

    # sklearn returns MI in nats (natural log base)
    I_ZY = mutual_info_regression(
        Z, y, n_neighbors=n_neighbors, random_state=random_state
    )
    I_ZY_total = float(np.sum(I_ZY))  # sum over Z dims approximates I(Z; y)

    I_ZX_Y = mutual_info_regression(
        ZX, y, n_neighbors=n_neighbors, random_state=random_state
    )
    I_ZX_Y_total = float(np.sum(I_ZX_Y))  # approximates I([Z,X]; y)

    cmi = I_ZX_Y_total - I_ZY_total
    # CMI can't be < 0 theoretically; clip tiny negatives due to estimation noise
    return max(0.0, cmi), len(y)

def _standardize_train_apply(
    X_train, X_val
):
    mu = X_train.mean(axis=0)
    sd = X_train.std(axis=0) + 1e-12
    return (X_train - mu) / sd, (X_val - mu) / sd, mu, sd

def _make_lag_matrix(x, lags):
    """Return [x_{t-l} for l in lags] stacked column-wise, aligned to t."""
    cols = []
    for l in range(1,lags+1):
        if l == 0:
            cols.append(x.copy())
        else:
            z = np.empty_like(x)
            z[:l] = x[0]
            z[l:] = x[:-l]
            cols.append(z)
    return np.column_stack(cols)

def train_test_lr(hr, imu):

    L = 30 
    H = 3

    train_end = int(0.7 * len(hr))

    hr_train, hr_test, _, _  = _standardize_train_apply(hr[:train_end], hr[train_end:])
    imu_train, imu_test, _, _ = _standardize_train_apply(imu[:train_end], imu[train_end:])

    y_train = _make_lag_matrix(hr_train, lags=L)
    x_train = _make_lag_matrix(imu_train, lags=L)

    import pdb 
    pdb.set_trace() 
    
    endo_lr = LinearRegression()
    exo_lr = LinearRegression()

    endo_lr.fit(y_train)
    exo_lr.fit(np.concat((y_train, x_train),axis=1))


In [190]:
sizes = [2,4,8]
starts = [2070, 4300, 1200]
ends = [2150, 4500, 1250]
for winsize, start, end in zip(sizes, starts, ends):
    outdict = process_data(winsize)
    train_test_lr(outdict["data_bpm_values"][0,:,0], outdict["imu"])
    print(conditional_mutual_info(outdict["data_bpm_values"][0,:,0], outdict["imu"]))
    #plot_data(winsize, start, end, outdict)
    #plot_scatter(outdict) 

> c:\users\cleme\appdata\local\temp\ipykernel_24324\3866619099.py(8)process_data()

dict_keys(['__header__', '__version__', '__globals__', 'id', 'notes', 'sternum', 'head', 'ankle', 'wrist'])
{'acc_x': {'fs': np.int32(128), 'descr': np.str_('x-axis of MEMS accelerometer'), 'v': array([0.92961694, 0.94211695, 0.93338305, ..., 0.936     , 0.9478826 ,
       0.9516174 ])}, 'acc_y': {'fs': np.int32(128), 'descr': np.str_('y-axis of MEMS accelerometer'), 'v': array([ 0.04476611,  0.04423389,  0.0507661 , ..., -0.04      ,
       -0.056     , -0.0563826 ])}, 'acc_z': {'fs': np.int32(128), 'descr': np.str_('z-axis of MEMS accelerometer'), 'v': array([-0.19676611, -0.17176611, -0.176     , ..., -0.152     ,
       -0.1481174 , -0.12876521])}, 'altitude': {'fs': np.float64(0.5), 'descr': np.str_('barometric altitude measurements, averaged in 8-second moving windows, with step size 2 seconds'), 'v': array([466.03814316, 466.06812859, 466.10313249, ..., 483.3344698 ,
       483.32486582, 483.3288